This runs confirmatory analyses in pretone_pLH data to check for the precue pupil effects for the section "Rule- and stimulus-based biases exhibited distinct physiological signatures"

# Data setup

In [1]:
#clear memory
rm(list=ls())

### this runs regressions for pretone_pLH choice-aligned pupil data

## LOADING data/libraries ##

#load libraries
library(lme4)
library(lmerTest)
library(car)
library(plyr)
library(dplyr)
library(ggplot2)
#library(afex)
library(emmeans)
emm_options(lmerTest.limit = Inf, lmer.df = "satterthwaite")


switch(Sys.info()[['sysname']],
       Windows = setwd(file.path(
         Sys.getenv('USERPROFILE'),'Dropbox/Goldlab/AuditoryPriors/data processing/pupil')),
       Darwin = setwd('~/Dropbox/Goldlab/AuditoryPriors/data processing/pupil')
)

#path to data files
switch(Sys.info()[['sysname']],
       Windows = DATA_OUT_PATH <- (paste0(
         Sys.getenv('USERPROFILE'),'/OneDrive/Goldlab/AuditoryPriors/cached data/')),
       Darwin = DATA_OUT_PATH <- '~/OneDrive/Goldlab/AuditoryPriors/cached data/'
)
pd_file= paste0(DATA_OUT_PATH,'pupil_data_pretonepLH_ds50_forR_08-Jun-2021.csv')
baseline_file = paste0(DATA_OUT_PATH,'pupil_data_pretonepLH_bl_forR_08-Jun-2021.csv')

#set temporal parameters 
#period in which incon-con is sig (corrected) in both choice regression and bias correlation
choice_period_ms.prior <- c(220,720) 

#load data
pd_df<-read.table(pd_file,sep=',', header=TRUE, stringsAsFactors=FALSE,na.strings = c('NaN'))
bl_df<-read.table(baseline_file,sep=',', header=TRUE, stringsAsFactors=FALSE,na.strings = c('NaN'))

head(pd_df)
head(bl_df)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Loading required package: carData


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:car’:

    recode


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




,trialN,trial_time_choice,trial_time_stimOn,trial_time_stimOff,pupilCblz,posXCbl,posYCbl,dataID,pupilCblz2,aSNR,isH,success,choice01,prior,pretoneLength,congruent,congruentpt
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,-2000,-260,-1760,-0.13820916,-5.787674,0.9855574,2,0.5896288,0.05,0,1,0,-2,3,1,0
2,1,-1980,-240,-1740,-0.13124094,-7.237302,-0.3603486,2,0.5965970,0.05,0,1,0,-2,3,1,0
3,1,-1960,-220,-1720,-0.12247279,-6.464236,-0.2966258,2,0.6053651,0.05,0,1,0,-2,3,1,0
4,1,-1940,-200,-1700,-0.11226265,-6.336257,0.7309437,2,0.6155753,0.05,0,1,0,-2,3,1,0
5,1,-1920,-180,-1680,-0.10108940,-5.805152,0.6163435,2,0.6267485,0.05,0,1,0,-2,3,1,0
6,1,-1900,-160,-1660,-0.08961575,-4.953268,0.1747674,2,0.6382222,0.05,0,1,0,-2,3,1,0


,trialN,GroupCount,pupilBL,posXBL,posYBL,dataID_block,pupilBL2,aSNR,isH,success,choice01,prior,pretoneLength,pretoneBias,ptC,congruent,congruentpt,dataID
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<int>,<int>,<int>
1,1,41,-0.5101309,1239.427,727.6366,2,-0.72783792,0.05,0,1,0,-2,3,0.3333333,HH,1,0,2
2,2,41,0.5813434,1230.868,713.3317,2,0.22185096,0.50,0,1,0,-2,14,0.8571429,HH,1,0,2
3,3,41,1.0496002,1227.105,696.5939,2,-1.98890946,0.50,0,1,0,-2,4,-1.0000000,LL,1,3,2
4,4,41,0.6929309,1210.072,702.9085,2,-0.07113367,0.50,0,1,0,-2,8,0.7500000,LH,1,2,2
5,6,41,1.4616907,1208.493,718.5256,2,-0.19761281,0.10,0,1,0,-2,9,-0.5555556,LL,1,3,2
6,7,41,1.0742042,1218.302,740.8134,2,-0.49286442,0.05,0,0,1,-2,9,0.7777778,HH,0,3,2


In [2]:
#cleanup
bl_df$GroupCount <- NULL

#set up variables/factors
my_simple2<-contr.treatment(2,base=2) - matrix(rep(1/2,2))

pd_df$congruentpt.f <- factor(pd_df$congruentpt,levels=c(3,2,1,0),
                            labels=c("con","con-incon","incon-con","incon"))
contrasts(pd_df$congruentpt.f) <- contr.sum(4)
contrasts(pd_df$congruentpt.f)
unique(pd_df[,c('congruentpt','congruentpt.f')])

pd_df$congruent.f <- factor(pd_df$congruent,levels=c(0,1),
                            labels=c("incongruent","congruent"))
contrasts(pd_df$congruent.f) <- contr.sum(2)
contrasts(pd_df$congruent.f)
unique(pd_df[,c('congruent','congruent.f')])

pd_df$isH.fs <- factor(pd_df$isH,levels=c(1,0),
                      labels=c("high","low"))
contrasts(pd_df$isH.fs) <- my_simple2
contrasts(pd_df$isH.fs)
pd_df$isH.f <- pd_df$isH.fs
contrasts(pd_df$isH.f) <- contr.sum(2)
contrasts(pd_df$isH.f)

#merge in baseline data
pd_df <- left_join(pd_df,bl_df[,c('dataID','prior','trialN','pupilBL2')],
                   by=c('dataID','prior','trialN'))

#get rid of missing before scaling vars (probably already done w/ new preproc)
pd_df <- pd_df[complete.cases(pd_df),]

#get a handle on descriptives to inform scaling
scalevars_stats <- summarise(pd_df,
                             m_bl=mean(pupilBL2,na.rm=T),
                             sd_bl=sd(pupilBL2,na.rm=T),
                             m_x=mean(posXCbl,na.rm=T),
                             sd_x=sd(posXCbl,na.rm=T),
                             m_y=mean(posYCbl,na.rm=T),
                             sd_y=sd(posYCbl,na.rm=T),
                             m_asnr=mean(aSNR,na.rm=T),
                             sd_asnr=sd(aSNR,na.rm=T),
)

#set up variables: centering/scaling
pd_df$zaSNR <- scale(pd_df$aSNR)
pd_df$blz <- scale(pd_df$pupilBL2,center=T,scale=F) #not scaling since already scaled
pd_df$posX <- scale(pd_df$posXCbl)
pd_df$posY <- scale(pd_df$posYCbl)
pd_df$zptlen <- scale(pd_df$pretoneLength)

con,1,0,0
con-incon,0,1,0
incon-con,0,0,1
incon,-1,-1,-1


,congruentpt,congruentpt.f
,<int>,<fct>
1,0,incon
163,3,con
394,2,con-incon
3489,1,incon-con


incongruent,1
congruent,-1


,congruent,congruent.f
,<int>,<fct>
1,1,congruent
942,0,incongruent


,1
high,0.5
low,-0.5


high,1
low,-1


# Correct trials, precue window

In [3]:
### CORRECT TRIALS, precue window ####
pd_choicedf.cpc <- subset(pd_df,success==1 & 
                            trial_time_choice >= choice_period_ms.prior[1] & 
                            trial_time_choice <= choice_period_ms.prior[2])

pd_choicedf.cpc.ave <- summarise(group_by(pd_choicedf.cpc,
                                          dataID,trialN,prior,
                                          zaSNR,congruent.f,congruentpt.f,isH.f,zptlen),
                                 pupilCblz2=mean(pupilCblz2),
                                 blz=mean(blz),#blz could be a grouping factor but this is fine
                                 posX=mean(posX),
                                 posY=mean(posY))

#set up vars for zero corr
pd_choicedf.cpc.ave[,c('congruentpt.f1','congruentpt.f2','congruentpt.f3')] <- 
  model.matrix(~1+pd_choicedf.cpc.ave$congruentpt.f,pd_choicedf.cpc.ave)[,2:4]
pd_choicedf.cpc.ave$isH.f1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$isH.f,pd_choicedf.cpc.ave)[,2]
pd_choicedf.cpc.ave$congruent.f1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$congruent.f,pd_choicedf.cpc.ave)[,2]
unique(pd_choicedf.cpc.ave[,c("congruentpt.f","congruentpt.f1","congruentpt.f2","congruentpt.f3")])
unique(pd_choicedf.cpc.ave[,c("isH.f","isH.f1")])
unique(pd_choicedf.cpc.ave[,c("congruent.f","congruent.f1")])

`summarise()` has grouped output by 'dataID', 'trialN', 'prior', 'zaSNR', 'congruent.f', 'congruentpt.f', 'isH.f'. You can override using the `.groups` argument.



congruentpt.f,congruentpt.f1,congruentpt.f2,congruentpt.f3
<fct>,<dbl>,<dbl>,<dbl>
incon,-1,-1,-1
con,1,0,0
con-incon,0,1,0
incon-con,0,0,1


isH.f,isH.f1
<fct>,<dbl>
low,-1
high,1


congruent.f,congruent.f1
<fct>,<dbl>
congruent,-1
incongruent,1


In [ ]:
#these have boundary issues, so iteratively reducing

choice.lm.zc.cpc <- lmer(pupilCblz2~
                           congruent.f + congruentpt.f + zaSNR +
                           isH.f + blz + posX + posY + zptlen + 
                           (1 + congruent.f1 + congruentpt.f1 + congruentpt.f2 + congruentpt.f3 + 
                              zaSNR + isH.f1 + blz + posX+ posY + zptlen||dataID),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

VarCorr(choice.lm.zc.cpc)

In [7]:
choice.lm.cpc.2 <- lmer(pupilCblz2~
                           congruent.f + congruentpt.f + zaSNR +
                           isH.f + blz + posX + posY + zptlen + 
                           (1 + congruent.f1 + congruentpt.f2 + 
                              zaSNR + isH.f1 + blz + posX + posY + zptlen|dataID),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular



 Groups    Name           Std.Dev. 
 dataID    (Intercept)    0.1695496
 dataID.1  congruent.f1   0.0346219
 dataID.2  congruentpt.f1 0.0000000
 dataID.3  congruentpt.f2 0.0085887
 dataID.4  congruentpt.f3 0.0000000
 dataID.5  zaSNR          0.0241081
 dataID.6  isH.f1         0.0523479
 dataID.7  blz            0.1516978
 dataID.8  posX           0.0657938
 dataID.9  posY           0.0993972
 dataID.10 zptlen         0.0420262
 Residual                 0.3240524

boundary (singular) fit: see ?isSingular



In [4]:
#FINAL
#FINAL MODEL
#drop congruentpt.f1/f3
choice.lm.zc.cpc.2 <- lmer(pupilCblz2~
                           congruent.f + congruentpt.f + zaSNR +
                           isH.f + blz + posX + posY + zptlen + 
                           (1 + congruent.f1 + congruentpt.f2 + 
                              zaSNR + isH.f1 + blz + posX + posY + zptlen||dataID),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

In [5]:
#FINAL
summary(choice.lm.zc.cpc.2)
anova(choice.lm.zc.cpc.2,type="II")

pairs(emmeans(choice.lm.zc.cpc.2,~congruent.f))

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: pupilCblz2 ~ congruent.f + congruentpt.f + zaSNR + isH.f + blz +  
    posX + posY + zptlen + (1 + congruent.f1 + congruentpt.f2 +  
    zaSNR + isH.f1 + blz + posX + posY + zptlen || dataID)
   Data: pd_choicedf.cpc.ave
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: 8512.2

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-12.3224  -0.6140  -0.0542   0.5702   9.7709 

Random effects:
 Groups   Name           Variance  Std.Dev.
 dataID   (Intercept)    2.875e-02 0.169550
 dataID.1 congruent.f1   1.199e-03 0.034622
 dataID.2 congruentpt.f2 7.377e-05 0.008589
 dataID.3 zaSNR          5.812e-04 0.024108
 dataID.4 isH.f1         2.740e-03 0.052348
 dataID.5 blz            2.301e-02 0.151698
 dataID.6 posX           4.329e-03 0.065794
 dataID.7 posY           9.880e-03 0.099397
 dataID.8 zptlen         1.766e-03 0.042026
 

,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
congruent.f,7.61261624,7.61261624,1,29.60299,72.494254,1.875270e-09
congruentpt.f,3.38635008,1.12878336,3,172.24627,10.749302,1.639486e-06
zaSNR,2.25020168,2.25020168,1,30.81402,21.428466,6.276590e-05
isH.f,0.04224423,0.04224423,1,31.96777,0.402288,5.304230e-01
blz,21.27462008,21.27462008,1,31.86263,202.596276,2.331247e-15
posX,2.85335177,2.85335177,1,31.85022,27.172210,1.084012e-05
posY,1.57662088,1.57662088,1,32.06818,15.014018,4.959583e-04
zptlen,21.16191458,21.16191458,1,30.37305,201.522992,6.009891e-15


 contrast                estimate     SE   df t.ratio p.value
 incongruent - congruent     0.14 0.0165 29.6   8.514  <.0001

Results are averaged over the levels of: congruentpt.f, isH.f 
Degrees-of-freedom method: satterthwaite 